<a href="https://colab.research.google.com/github/barthas1985/Stock_indicators_Warsaw_Stock_Exchange/blob/main/02_04_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
!apt-get install mysql-server > /dev/null
!service mysql start
!mysql -e "ALTER USER 'root'@'localhost' IDENTIFIED WITH mysql_native_password BY 'root'"
!pip -q install PyMySQL

 * Starting MySQL database server mysqld
   ...done.
ERROR 1045 (28000): Access denied for user 'root'@'localhost' (using password: NO)


In [57]:
!pip install SQLAlchemy-Utils

In [58]:
from sqlalchemy import create_engine
from sqlalchemy_utils import create_database, database_exists 

In [59]:
engine = create_engine('sqlite:///mydb.db', echo = True) # Śczieżka do bazy danych, podstawowa ścieżka do bazy danych

In [60]:
if not database_exists(engine.url):
  create_database(engine.url)

CREATE SESSION - for ORM

In [61]:
from sqlalchemy.orm import sessionmaker as s

In [62]:
Session = s(bind = engine)

Define Table

In [63]:
from sqlalchemy.orm import declarative_base as db
from sqlalchemy import Column, Integer, String, Float

In [64]:
Base = db()

In [65]:
class User(Base):
  __tablename__ = 'users'
  id = Column(Integer, primary_key = True)
  name =Column(String)
  fullname = Column(String)
  nickname = Column(String)

In [66]:
session1 = Session()

In [67]:
session1.new

IdentitySet([])

In [68]:
session1.add(
    User(name = 'James', fullname = 'James Bond', nickname = '007')
)

In [69]:
session1.new

IdentitySet([<__main__.User object at 0x7f915fc8e510>])

In [70]:
bond = User(name = 'James', fullname = 'James Bond', nickname = '007')
bond.name

'James'

#Formatowanie obiektów klasy metodą __repr__

In [71]:
class User(Base):
  __tablename__ = 'users_new'
  id = Column(Integer, primary_key = True)
  name =Column(String)
  fullname = Column(String)
  nickname = Column(String)
  def __repr__(self):
    return f'User(name = {self.name}, fullname = {self.fullname}, nickname = {self.nickname}'
    


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.User, and will be replaced in the string-lookup table.
  """Entry point for launching an IPython kernel.


In [72]:
new_user = User(name = 'Bartek', fullname= 'Bartek Gajek', nickname = 'Bart')
new_user

User(name = Bartek, fullname = Bartek Gajek, nickname = Bart

In [73]:
session2 = Session()


In [74]:
session2.add(
    User(name = 'James', fullname = 'James Bond', nickname = '007')
)

In [75]:
session2.new

IdentitySet([User(name = James, fullname = James Bond, nickname = 007])

In [76]:
session2.add_all(
    [
     User(name = 'user1', fullname = 'User 1 USer', nickname = '005'),
     User(name = 'Johnny', fullname = 'Johnny Cash', nickname = '$$')
    ]
)

In [77]:
session2.new

IdentitySet([User(name = James, fullname = James Bond, nickname = 007, User(name = user1, fullname = User 1 USer, nickname = 005, User(name = Johnny, fullname = Johnny Cash, nickname = $$])

In [78]:
Base.metadata.create_all(engine)

2022-04-02 08:43:51,776 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-02 08:43:51,783 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2022-04-02 08:43:51,788 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-02 08:43:51,794 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users_new")
2022-04-02 08:43:51,796 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-02 08:43:51,797 INFO sqlalchemy.engine.Engine COMMIT


In [79]:
session2.commit()

2022-04-02 08:43:51,808 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-02 08:43:51,816 INFO sqlalchemy.engine.Engine INSERT INTO users_new (name, fullname, nickname) VALUES (?, ?, ?)
2022-04-02 08:43:51,818 INFO sqlalchemy.engine.Engine [generated in 0.00188s] ('James', 'James Bond', '007')
2022-04-02 08:43:51,820 INFO sqlalchemy.engine.Engine INSERT INTO users_new (name, fullname, nickname) VALUES (?, ?, ?)
2022-04-02 08:43:51,823 INFO sqlalchemy.engine.Engine [cached since 0.006741s ago] ('user1', 'User 1 USer', '005')
2022-04-02 08:43:51,829 INFO sqlalchemy.engine.Engine INSERT INTO users_new (name, fullname, nickname) VALUES (?, ?, ?)
2022-04-02 08:43:51,830 INFO sqlalchemy.engine.Engine [cached since 0.01443s ago] ('Johnny', 'Johnny Cash', '$$')
2022-04-02 08:43:51,832 INFO sqlalchemy.engine.Engine COMMIT


Odczyt z bazy danych

In [80]:
from sqlalchemy import text

In [84]:
query = """
SELECT * 
FROM users_new
"""

In [85]:
result = session2.execute(text(query))

2022-04-02 08:51:05,216 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-02 08:51:05,221 INFO sqlalchemy.engine.Engine 
SELECT * 
FROM users_new

2022-04-02 08:51:05,224 INFO sqlalchemy.engine.Engine [generated in 0.00295s] ()


In [86]:
for row in result:
  print(row)

(1, 'James', 'James Bond', '007')
(2, 'user1', 'User 1 USer', '005')
(3, 'Johnny', 'Johnny Cash', '$$')


In [87]:
result = engine.execute(text(query))


2022-04-02 08:52:11,591 INFO sqlalchemy.engine.Engine 
SELECT * 
FROM users_new

2022-04-02 08:52:11,594 INFO sqlalchemy.engine.Engine [cached since 66.37s ago] ()


In [88]:
for row in result:
  print(row)

(1, 'James', 'James Bond', '007')
(2, 'user1', 'User 1 USer', '005')
(3, 'Johnny', 'Johnny Cash', '$$')
